In [6]:
import typing as tp
from pathlib import Path
from functools import partial
from dataclasses import dataclass, field
import matplotlib

import pandas as pd
import pyctcdecode
import numpy as np
from tqdm.notebook import tqdm

import librosa

import pyctcdecode
import kenlm
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ProcessorWithLM, Wav2Vec2ForCTC
from bnunicodenormalizer import Normalizer

import cloudpickle as cpkl

In [7]:
ROOT = Path.cwd().parent
print(ROOT)
INPUT = ROOT / "input"
DATA = INPUT / "bengaliai-speech"
TRAIN = DATA / "train_mp3s"
TRAIN_WAV = INPUT / "train_wavs"
TEST = DATA / "test_mp3s"
NISQA = Path.cwd() / "nisqa"

SAMPLING_RATE = 16_000

/home/ding2/tf-practice/kaggle-Bengali.AI_Speech-Recognition


In [16]:
# TRAIN_WAV の中にあるファイルを100個取り出し、../input/train_wavs_100 に保存する
import os
import shutil
import random

os.mkdir(INPUT / "train_wavs_100")

files = os.listdir(TRAIN_WAV)

for i in range(1000):
    file = files[i]
    shutil.copyfile(TRAIN_WAV / file, INPUT / "train_wavs_100" / file)

In [26]:
%run ./NISQA/run_predict.py --mode predict_dir --pretrained_model ./NISQA/weights/nisqa.tar --data_dir ../input/train_wavs_1000 --num_workers 4 --bs 20 --output_dir ./results

Device: cuda
Model architecture: NISQA_DIM
Loaded pretrained model from ./NISQA/weights/nisqa.tar
# files: 1000
---> Predicting ...


/home/ding2/anaconda3/envs/tf-practice/lib/python3.10/site-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)
/home/ding2/anaconda3/envs/tf-practice/lib/python3.10/site-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)
/home/ding2/anaconda3/envs/tf-practice/lib/python3.10/site-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)
/home/ding2/anacond

             deg  mos_pred  noi_pred  dis_pred  col_pred  loud_pred   model
000005f3362c.wav  1.186517  1.974640  2.046895  2.519453   3.803557 NISQAv2
00001dddd002.wav  2.432569  3.634542  2.492738  2.933400   3.484299 NISQAv2
00001e0bc131.wav  3.460908  4.089850  3.910269  3.362554   3.865279 NISQAv2
000024b3d810.wav  4.643417  4.261642  4.742973  4.624074   4.678082 NISQAv2
000028220ab3.wav  2.385320  3.697676  2.107449  2.628804   2.052197 NISQAv2
00002b0c8953.wav  1.483584  2.752193  2.495244  2.488301   2.958117 NISQAv2
00003147472f.wav  2.711868  3.614980  3.197957  2.504451   3.613499 NISQAv2
000043f5b62d.wav  2.923751  4.128201  2.881648  3.475107   4.028925 NISQAv2
00004ed6485a.wav  2.317027  3.504913  2.934121  2.684947   2.844898 NISQAv2
000057f5f4a3.wav  2.276857  3.085186  2.967828  2.876264   3.643267 NISQAv2
00006b6c9788.wav  1.851597  3.710734  4.161140  2.658181   1.677719 NISQAv2
000070ccd82f.wav  1.555002  3.367154  2.963323  2.635464   1.605137 NISQAv2
0000900974a8

In [12]:
%run ./NISQA/run_predict.py --mode predict_file --pretrained_model ./NISQA/weights/nisqa.tar --deg ../input/train_wavs/000005f3362c.wav --output_dir ./results

Device: cuda
Model architecture: NISQA_DIM
Loaded pretrained model from ./NISQA/weights/nisqa.tar
---> Predicting ...
             deg  mos_pred  noi_pred  dis_pred  col_pred  loud_pred   model
000005f3362c.wav  1.186509  1.974641  2.046838  2.519474   3.803517 NISQAv2


/home/ding2/anaconda3/envs/tf-practice/lib/python3.10/site-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


In [ ]:
test = pd.read_csv(DATA / "sample_submission.csv", dtype={"id": str})